# Interactive visualisation of a subnetwork in the connectome

If there's a relatively small number of neurons you're interested in you can visualise their connections in your browser and drag around the neurons to get a feel for what the network looks like.

In [1]:
#%matplotlib notebook
import pandas
import utils as u

In [2]:
from vnc_networks.vnc_networks.connectome_reader import MANC
from vnc_networks.vnc_networks.connections import Connections
import matplotlib.pyplot as plt
import numpy as np
#import utils



# Load the connections in the VNC from the MANC dataset
CR = MANC('v1.2') # Connectome Reader
vnc = Connections(CR = CR) # Connections object, this takes some time

Attribute class_1 not found in the graph. Adding it.


I chose to show the connections from DNg11 because I know it doesn't project to so many neurons in the VNC so the network will be easy to visualise

In [10]:
# Get the motor neurons for a specific leg, here the right front leg
side = 'RHS' # right hand side, can also be 'LHS'
target = 'fl' # front leg, can also be 'ml' or 'hl'

# Find an interesting input to a network, here a group of descending neurons named MDN
input_neurons = vnc.get_neuron_ids(
    {
        'class_1' : 'descending', # descending neuron
        #'type': 'DNg11', # a specific name
    }
)

motor_neurons = vnc.get_neuron_ids( # intersection (AND) logic 
    {
        'class_1': 'motor', # class_1 is the main neuron class such as 'sensory', 'descending' etc.
        'class_2': target, # subclass, its nature depends on class_1 and dataset
        'side': side,
    }
)

ascending_neurons = vnc.get_neuron_ids(
    {
        'class_1' : 'ascending',

    })
print(f"Found {len(input_neurons)} descending neurons")
print(f"Found {len(ascending_neurons)} ascending neurons")
print(f"Found {len(motor_neurons)} motor neurons for the {side} {target} leg")

ValueError: ConnectomeReader::sna().                    The attribute area is not defined in manc.

We get connections:

* from DNg11
* to motor neurons in the right front leg
* with up to 2 hops

In [27]:
utils.plot_cluster_similarity_matrix(vnc,motor_neurons)

<IPython.core.display.Javascript object>

Weights of neurons connections : (positive = exitacory , negative = inhibitory)

{'acetylcholine': 1, 'gaba': -1, 'glutamate': -1, 'unknown': 0, None: 0, nan: 0}

In [6]:
weights = vnc.get_nt_weights()
print(weights)

{'acetylcholine': 1, 'gaba': -1, 'glutamate': -1, 'unknown': 0, None: 0, nan: 0}


In [40]:
uid1 = vnc.get_uids_from_bodyid(26209)
uid2 = vnc.get_uids_from_bodyid(24966)
print(uid1,uid2)

[593] [723]


In [38]:
nb_synapses = vnc.get_nb_synapses(10294,11706,"body_id")
print(nb_synapses)

25


In [4]:
# Keep only the connections that create a path from source to target
subnetwork = vnc.subgraph_from_paths( # copy operation
    source = input_neurons,
    target = motor_neurons,
    n_hops = 2, # within 2 hops, i.e. only 1 interneuron
    keep_edges = 'direct', # keep the connections between the interneurons
    # as well, but not between source neurons or between target neurons
    # can also be 'direct' (only direct paths) or 'all' (all connections between
    # recruited nodes)
)
#print(f"Found {len(subnetwork)} connections to front motor leg")


Attribute class_1 not found in the graph. Adding it.


In [14]:
name = subnetwork.get_node_attribute(subnetwork.get_uids_from_bodyid(10322), "class_1")
print(name)

['ascending']


In [5]:
#Trying stuff
#data = subnetwork.get_dataframe()
#graph = vnc.paths_length_n(2,source=input_neurons,target=motor_neurons)
#nodes = subnetwork.get_nodes("body_id")
u.show_data_connections(subnetwork)
#print(data)
#edges = graph.edges
#nodes = graph.nodes
#print(nodes)
#(testdata).applymap(_->_)
#display(testdata)
#print(nodes)

Attribute name not found in the graph. Adding it.


,source_body_ids,source_name,source_class,target_body_id,target_name,target_class,syn_count,eff_weights
0,10294,[AN07B003],[ascending],11706,[MNfl39],[motor],25,25
1,10322,[AN07B003],[ascending],11226,[MNfl35],[motor],13,13
2,10322,[AN07B003],[ascending],11706,[MNfl39],[motor],38,38
3,10322,[AN07B003],[ascending],12628,[MNfl55],[motor],9,9
4,10322,[AN07B003],[ascending],15446,[MNfl52],[motor],10,10
5,10322,[AN07B003],[ascending],18440,[MNfl07],[motor],58,58
6,10322,[AN07B003],[ascending],19751,[MNfl52],[motor],28,28
7,10322,[AN07B003],[ascending],19886,[MNfl12],[motor],32,32
8,10322,[AN07B003],[ascending],20673,[MNfl12],[motor],8,8
9,23688,[DNxn014],[descending],10322,[AN07B003],[ascending],5,-5


Now we visualise the subnetwork by giving it to the `display_interactive_graph` function.

You can hover over the neurons or connections for more information. Basically, 

* the dark blue neurons are DNs
* the red connections are excitatory
* blue connections are inhibitory
* pink connections are glutamate (not really sure if it's excitatory or inhibitory)

In [12]:
# visualise the result as an interactive graph
# running this code should cause the visualisation to popup in your browser
from vnc_networks.vnc_networks.utils.nx_design import display_interactive_graph

display_interactive_graph(subnetwork)

Attribute class_2 not found in the graph. Adding it.
Attribute nt_type not found in the graph. Adding it.
Attribute side not found in the graph. Adding it.
Attribute neuropil not found in the graph. Adding it.
Attribute hemilineage not found in the graph. Adding it.


It should look like this

![DNg11 network](DNg11%20network.png)

We could also use this on the MDN to motor neuron network but it's huge / very hard to make sense of.

![MDN network](MDN%20network.png)